In [1]:
import pandas as pd
import geopandas

In [17]:
county_csv = pd.read_csv('src/data/mn_demographics_history.csv')
county_csv.columns

Index(['statefp', 'county', 'fips', 'population2018', 'classification',
       'dVotes_2012', 'dPct_2012', 'rVotes_2012', 'rPct_2012', 'votes_2012',
       'd_two_party_2012', 'r_two_party_2012', 'partisanVotes_2012',
       'winner_2012', 'dVotes_2016', 'dPct_2016', 'rVotes_2016', 'rPct_2016',
       'votes_2016', 'd_two_party_2016', 'r_two_party_2016',
       'partisanVotes_2016', 'winner_2016', 'flipped', 'cpvi_score',
       'cpvi_rating', 'turnout_2012', 'turnout_2016', 'turnout_2018',
       'd_gov_2018', 'd_gov_pct_2018', 'r_gov_2018', 'r_gov_pct_2018',
       'votes_gov_2018', 'winner_2018'],
      dtype='object')

In [3]:
# Bring in TIGER shapefile for data
county_tiger = geopandas.read_file('/Users/Coreymj/Documents/GIS/usa/tl_2019_us_county/tl_2019_us_county.shp')

In [4]:
county_tiger_mn = county_tiger[county_tiger['STATEFP'] == '27'].copy()
county_tiger_mn.GEOID = county_tiger_mn.GEOID.astype('int64')
county_tiger_mn.sort_values(['NAME']).head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
289,27,001,00663198,27001,Aitkin,Aitkin County,06,H1,G4020,None,None,None,A,4718433588,449373340,+46.6024464,-093.4197600,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1612,27,003,00659447,27003,Anoka,Anoka County,06,H1,G4020,378,33460,None,A,1092880512,62442714,+45.2741098,-093.2427232,"POLYGON ((-93.51099 45.29935, -93.51097 45.300..."
772,27,005,00659448,27005,Becker,Becker County,06,H1,G4020,None,None,None,A,3406052328,336954677,+46.9376289,-095.7417568,"POLYGON ((-96.17430 46.89165, -96.17430 46.891..."
905,27,007,00659449,27007,Beltrami,Beltrami County,06,H1,G4020,None,13420,None,A,6487154450,1426287781,+47.8795077,-095.0050500,"POLYGON ((-95.59382 48.19455, -95.59382 48.194..."
1705,27,009,00659450,27009,Benton,Benton County,06,H1,G4020,378,41060,None,A,1057483359,12286988,+45.7012249,-094.0014397,"POLYGON ((-94.24770 45.73522, -94.24784 45.735..."


In [18]:
county_out = county_csv.copy()[[
    'statefp',
    'county',
    'fips',
    'population2018',
    'classification',
    'dPct_2016',
    'rPct_2016'
]]

county_out.fips = county_out.fips.astype('int64')
county_out.head()

,statefp,county,fips,population2018,classification,dPct_2016,rPct_2016
0,27,Aitkin,27001,15896,rural,0.341,0.601
1,27,Anoka,27003,357851,suburban,0.410,0.507
2,27,Becker,27005,34420,rural,0.305,0.637
3,27,Beltrami,27007,46834,rural,0.408,0.506
4,27,Benton,27009,40452,rural,0.283,0.646


In [19]:
county_tiger_mn[[
        'GEOID',
        'ALAND'
    ]].sort_values(['GEOID'])

,GEOID,ALAND
289,27001,4718433588
1612,27003,1092880512
772,27005,3406052328
905,27007,6487154450
1705,27009,1057483359
...,...,...
620,27165,1126526388
1672,27167,1945079889
878,27169,1621638207
633,27171,1712289572


In [20]:
county_out = county_out.merge(
    county_tiger_mn[[
        'GEOID',
        'ALAND'
    ]],
    how="left",
    left_on="fips",
    right_on="GEOID"
)
county_out.head()

,statefp,county,fips,population2018,classification,dPct_2016,rPct_2016,GEOID,ALAND
0,27,Aitkin,27001,15896,rural,0.341,0.601,27001,4718433588
1,27,Anoka,27003,357851,suburban,0.410,0.507,27003,1092880512
2,27,Becker,27005,34420,rural,0.305,0.637,27005,3406052328
3,27,Beltrami,27007,46834,rural,0.408,0.506,27007,6487154450
4,27,Benton,27009,40452,rural,0.283,0.646,27009,1057483359


In [21]:
county_out['pop_density_2018'] = county_out['population2018'] / (county_out['ALAND'] / 2589988.1103)
county_out.sort_values('pop_density_2018', ascending=False).head()

,statefp,county,fips,population2018,classification,dPct_2016,rPct_2016,GEOID,ALAND,pop_density_2018
84,27,Ramsey,27123,552232,urban,0.657,0.263,27123,394308034,3627.301984
81,27,Hennepin,27053,1261104,urban,0.638,0.285,27053,1434450857,2276.999836
1,27,Anoka,27003,357851,suburban,0.410,0.507,27003,1092880512,848.061453
80,27,Dakota,27037,428558,suburban,0.482,0.436,27037,1456983368,761.820724
86,27,Washington,27163,261512,suburban,0.470,0.451,27163,995618866,680.293427


In [22]:
mn_incomes = pd.read_csv('src/data/countyMHHI2018.csv')
mn_incomes.head()

,geoid,name,estimate,moe
0,27001,"Aitkin County, Minnesota",45989,2244
1,27003,"Anoka County, Minnesota",79923,1132
2,27005,"Becker County, Minnesota",59278,1882
3,27007,"Beltrami County, Minnesota",47410,2260
4,27009,"Benton County, Minnesota",56357,2775


In [23]:
county_out = county_out.merge(
    mn_incomes[[
        'geoid',
        'estimate'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid"
)
county_out.rename(columns={'estimate': 'median_income'}, inplace=True)

In [24]:
county_out.to_json('src/data/mn_demographics_history.json', orient='records')